# Ellie's Transcript Data Processing
This code is meant to take in a `.csv` file of each participant's transcript from his/her interview with Ellie, and clean it up. 

First, we remove all unnecessary information from the `.csv` file.

1) Delete the timestamps from each transcript (i.e. the first 2 columns of the file) 

2) Delete any dialog from Ellie

3) Combine the remaining transcript into a single cell 

4) Insert the combined transcript into a global dataframe with the label, togteher with other transcripts

In [95]:
import pandas as pd 
import os
import ntpath

filepath_labels = "/Users/glendawee/Desktop/full_dataset_excl_interrupted_sessions.csv" #insert filepath to labels here
df_labels = pd.read_csv(filepath_labels, index_col = "Participant_ID")

df_labels.head(10)


,PHQ_Binary,PHQ_Score
Participant_ID,,
300,0,2
301,0,3
302,0,4
303,0,0
304,0,6
305,0,7
306,0,0
307,0,4
308,1,22


In [98]:
filepath_transcript = "/Users/glendawee/Desktop/300_P/300_TRANSCRIPT.csv" #insert file path to transcript here 
filename = ntpath.basename(filepath) # if this doesn't work, try `os.path.basename(filepath)`

participant_no = int(filename[0:3])
PHQ_Binary = df_labels.loc[308]["PHQ_Binary"]
PHQ_Score = df_labels.loc[308]["PHQ_Score"]

print(PHQ_Binary, PHQ_Score)

1 22


In [99]:
df_transcript = pd.read_csv(filepath_transcript, sep = '\t') 

column_list = df.columns.tolist()
#print(column_list)

df_dropped_columns = df_transcript.drop(['start_time', 'stop_time'], axis = 1) #drop first 2 columns
df_dropped_ellie = df_dropped_columns.set_index('speaker').drop(index = ['Ellie']) #drop anything Ellie says, she's not important
df_cleaned = df_dropped_ellie.reset_index() 

In [100]:
#concatenating participant's transcript into a single string

transcript_concat = '' 

for index, row in df_cleaned.iterrows():
    if (index == 0):
        transcript_concat += (row['value'])
    else:
        transcript_concat += ' ' + (row['value'])
    

print(transcript_concat)
    

good atlanta georgia um my parents are from here um i love it i like the weather i like the opportunities um yes um it took a minute somewhat easy congestion that's it um i took up business and administration uh yeah i am here and there i'm on a break right now but i plan on going back in the uh next semester uh probably to open up my own business no um no specific reason i just don't travel a lot i'm pretty local once a year can you be a little bit more specific no answer i like reading books i enjoy i enjoy cooking um exercising is great i'm i'm i'm pretty good at it um yeah um probably about two weeks ago uh frustrated um i don't like bias um i don't like um when someone says they're gonna do something and they don't uh somewhat friendship i like to play sports i enjoy uh going out with friends and family playing games grandparents parents um yeah i mean they've always given me great advice they've always kept it real real close i would say going to college right after high school w

In [104]:
#putting participant's transcript into a global dataframe
column_names_final = ["Participant_ID", "Transcript", "PHQ_Score", "PHQ_Binary"]
df_final = pd.DataFrame(columns = column_names_final)
print(df_final)

new_row = {"Participant_ID": participant_no, "Transcript": transcript_concat, "PHQ_Score": PHQ_Score, "PHQ_Binary": PHQ_Binary}
df_final = df_final.append(new_row, ignore_index = True)

df_final.head(10)

Empty DataFrame
Columns: [Participant_ID, Transcript, PHQ_Score, PHQ_Binary]
Index: []


,Participant_ID,Transcript,PHQ_Score,PHQ_Binary
0,300,good atlanta georgia um my parents are from he...,22,1
